### Import Libraries

In [1]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import customtkinter as ctk 
import tkinter.messagebox as tkmb 
import tkinter as tk


Login Page

In [2]:
import customtkinter as ctk 
import tkinter.messagebox as tkmb 



# Selecting GUI theme - dark, light , system (for system default) 
ctk.set_appearance_mode("dark") 

# Selecting color theme - blue, green, dark-blue 
ctk.set_default_color_theme("blue") 

app = ctk.CTk() 
app.geometry("400x400") 
app.title("Computer Vision Based Facemask Detection with Alertsystem") 


def login(): 

	username = "@user"
	password = "1234"
	new_window = ctk.CTkToplevel(app) 

	new_window.title("Computer Vision Based Face-Mask Detection") 

	new_window.geometry("350x150") 

	if user_entry.get() == username and user_pass.get() == password: 
		ctk.CTkLabel(new_window,text="....Starting the Streaming !!").pack() 
		tkmb.showinfo(title="Login Successful",message="You have logged in Successfully"
				)
		app.destroy()
	elif user_entry.get() == username and user_pass.get() != password: 
		tkmb.showwarning(title='Wrong password',message='Please check your password') 
	elif user_entry.get() != username and user_pass.get() == password: 
		tkmb.showwarning(title='Wrong username',message='Please check your username') 
	else: 
		tkmb.showerror(title="Login Failed",message="Invalid Username and password") 



label = ctk.CTkLabel(app,text="Please Login to Start the Streaming") 

label.pack(pady=20) 


frame = ctk.CTkFrame(master=app) 
frame.pack(pady=20,padx=40,fill='both',expand=True) 

label = ctk.CTkLabel(master=frame,text='Computer Vision login Credentials') 
label.pack(pady=12,padx=10) 


user_entry= ctk.CTkEntry(master=frame,placeholder_text="Username") 
user_entry.pack(pady=12,padx=10) 

user_pass= ctk.CTkEntry(master=frame,placeholder_text="Password",show="*") 
user_pass.pack(pady=12,padx=10) 


button = ctk.CTkButton(master=frame,text='Login',command=login) 
button.pack(pady=12,padx=10) 

checkbox = ctk.CTkCheckBox(master=frame,text='Remember Me') 
checkbox.pack(pady=12,padx=10)


app.mainloop()



### Upload Alarm Sound

In [3]:
from pygame import mixer
mixer.init()
sound = mixer.Sound('new.wav')

pygame 2.5.2 (SDL 2.28.3, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Image Pre-Processing

In [4]:
def mask_detection_prediction(frame, faceNet, maskNet):

    # find the dimension of frame and construct a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # create a empty list which'll store list of faces,face location and prediction
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        
        # find the confidence or probability associated with the detection
        confidence = detections[0, 0, i, 2]

        # filter the strong detection [confidence > min confidence(let 0.5)]
        if confidence > 0.5:

            # find starting and ending coordinates of boundry box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # make sure bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # append the face and bounding boxes to their respective lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding prediction
    return (locs, preds)


### Load Caffe Model
Caffe (Convolutional Architecture for Fast Feature Embedding) is a deep learning framework that allows users to create image classification and image segmentation models. It is a Caffe model which is based on the Single Shot-Multibox Detector (SSD) and uses ResNet-10 architecture as its backbone. It was introduced post OpenCV 3.3 in its deep neural network module.

In [5]:
# load our serialized face detector model from disk
from os.path import dirname, join

prototxtPath = join("face_detector", "deploy.prototxt")
weightsPath = join("face_detector", "res10_300x300_ssd_iter_140000.caffemodel")

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("fmd_model.h5")

### Face Detection on Live Camera

In [6]:
# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
# frame color
# root = tk.Tk()
# frame.config(background="dark")

# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = mask_detection_prediction(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        if mask>withoutMask:

            label = "Mask"
            color = (0, 255, 0)
            print("Normal")
        else:
            label = "No Mask"
            color = (0, 0, 255)
            sound.play()
            print("Alert!!!")

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output frame
        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("!!! Computer Vision Based Facemask Detection Surveillance System !!! ", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
1/1 [==============================] - 1s 1s/step
Normal
1/1 [==============================] - 0s 75ms/step
Alert!!!
1/1 [==============================] - 0s 64ms/step
Normal
1/1 [==============================] - 0s 58ms/step
Alert!!!
1/1 [==============================] - 0s 67ms/step
Alert!!!
1/1 [==============================] - 0s 62ms/step
Alert!!!
1/1 [==============================] - 0s 59ms/step
Alert!!!
1/1 [==============================] - 0s 76ms/step
Alert!!!
1/1 [==============================] - 0s 60ms/step
Alert!!!
1/1 [==============================] - 0s 61ms/step
Alert!!!
1/1 [==============================] - 0s 65ms/step
Alert!!!
1/1 [==============================] - 0s 60ms/step
Alert!!!
1/1 [==============================] - 0s 65ms/step
Alert!!!
1/1 [==============================] - 0s 57ms/step
Alert!!!
1/1 [==============================] - 0s 58ms/step
Alert!!!
1/1 [==============================] - 0s 60ms/step
Normal
